In [26]:
import pandas as pd
import numpy as np
import datetime as dt
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_QyjYwdj.csv")
campaign_data = pd.read_csv("campaign_data.csv")
coupon_item_mapping = pd.read_csv("coupon_item_mapping.csv")
customer_demographics = pd.read_csv("customer_demographics.csv")
customer_transaction_data = pd.read_csv("customer_transaction_data.csv")
item_data = pd.read_csv("item_data.csv")
sample_submission = pd.read_csv("sample_submission_2s8l9nF.csv")

In [3]:
customer_transaction_data.date = pd.to_datetime(customer_transaction_data.date)
campaign_data.start_date = pd.to_datetime(campaign_data.start_date, format = "%d/%m/%y")
campaign_data.end_date = pd.to_datetime(campaign_data.end_date, format = "%d/%m/%y")

In [4]:
campaign_data['campaign_duration'] = (campaign_data['end_date'] - campaign_data['start_date']).dt.days
campaign_data['campaign_type'] = campaign_data['campaign_type'].map(lambda x : 1 if x == 'Y' else 0)
customer_demographics['marital_status'] = customer_demographics['marital_status'].astype(str).map(lambda x : 1 if x == 'Married' else 0 if x == 'Single' else -1)
customer_demographics['age_range'] = customer_demographics['age_range'].map(lambda x : int(x.replace("+", "").split("-")[0]))
customer_demographics['no_of_children'] = customer_demographics['no_of_children'].fillna(0).astype(str).map(lambda x : int(x.replace("+", "")))
customer_demographics['family_size'] = customer_demographics['family_size'].fillna(0).astype(str).map(lambda x : int(x.replace("+", "")))
customer_demographics['age_range_by_income_bracket'] = customer_demographics['age_range'] / customer_demographics['income_bracket']

In [5]:
train['campaign_unique_coupon'] = train[['campaign_id', 'coupon_id']].groupby('campaign_id').transform('nunique')
train['campaign_unique_customer'] = train[['campaign_id', 'customer_id']].groupby('campaign_id').transform('nunique')

test['campaign_unique_coupon'] = test[['campaign_id', 'coupon_id']].groupby('campaign_id').transform('nunique')
test['campaign_unique_customer'] = test[['campaign_id', 'customer_id']].groupby('campaign_id').transform('nunique')

In [6]:
customer_transaction_data = customer_transaction_data.merge(item_data, on = 'item_id', how = 'left').merge(
    coupon_item_mapping, on = 'item_id', how = 'left')

In [7]:
campaign_transaction_data = []
campaign_cust_coupon_data = []
for row in tqdm(campaign_data.itertuples()):
    df = customer_transaction_data[(customer_transaction_data.date < row.start_date)]
    df['selling_price_per_item'] = df.selling_price/df.quantity
    df['other_discount_per_item'] = df.other_discount/df.quantity
    df['coupon_discount_per_item'] = df.coupon_discount/df.quantity
    df['total_discount'] = -1 * (df['other_discount'] + df['coupon_discount'])
    df['price_before_discount'] = df['selling_price'] + df['total_discount']
    df['total_discount_perc'] = df['total_discount'] / df['price_before_discount']
    df['item_best_deal'] = df[['item_id', 'selling_price_per_item']].groupby('item_id').cummin()
    df['item_curr_deal_vs_best_deal'] = (df['selling_price_per_item'] - df['item_best_deal'])*df.quantity

    df1 = df.copy()
    df = df.drop_duplicates(subset = [c for c in df.columns if c != "coupon_id"])
    campaign_customer_features = df[['customer_id', 'date']].groupby('customer_id').count().reset_index().rename(
                                                            columns = {'date' : 'campaign_customer_transaction_count'})
    campaign_customer_features['campaign_id'] = row.campaign_id
    campaign_customer_features['campaign_total_spend'] = df[['customer_id', 'selling_price']].groupby(
                                                                            'customer_id').sum().reset_index()['selling_price']
    campaign_months = set([(row.start_date + dt.timedelta(days = i)).month for i in range(row.campaign_duration)])
    
    campaign_customer_features['campaign_total_saving'] = df[['customer_id', 'total_discount']].groupby(
                                                                            'customer_id').sum().reset_index()['total_discount']
    campaign_customer_features['campaign_coupon_saving'] = df[['customer_id', 'coupon_discount']].groupby(
                                                                        'customer_id').sum().reset_index()['coupon_discount']
    campaign_customer_features['campaign_other_saving'] = df[['customer_id', 'other_discount']].groupby(
                                                                            'customer_id').sum().reset_index()['other_discount']
    cust_campaign_month_coupon_saving = df[df.date.dt.month.isin(campaign_months)][['customer_id', 'coupon_discount']].groupby(
                                            'customer_id').sum()['coupon_discount'].to_dict()
    campaign_customer_features['cust_campaign_month_coupon_saving'] = campaign_customer_features.customer_id.map(cust_campaign_month_coupon_saving)
    days_since_last_coupon_redemption = df[df.coupon_discount < 0][['customer_id', 'date']].groupby('customer_id'
                                                                                                   ).max().date.to_dict()
    campaign_customer_features['campaign_days_since_last_coupon_redemption'] = campaign_customer_features.customer_id.map(
                                                                                            days_since_last_coupon_redemption)
    campaign_customer_features['campaign_days_since_last_coupon_redemption'] = campaign_customer_features[
                                    'campaign_days_since_last_coupon_redemption'].map(lambda x : (row.start_date - x).days)

    days_since_last_transaction = df[['customer_id', 'date']].groupby('customer_id').max().date.to_dict()
    campaign_customer_features['campaign_days_since_last_transaction'] = campaign_customer_features.customer_id.map(
                                                                                                    days_since_last_transaction)
    campaign_customer_features['campaign_days_since_last_transaction'] = campaign_customer_features[
                                            'campaign_days_since_last_transaction'].map(lambda x : (row.start_date - x).days)

    campaign_customer_features['campaign_total_saving_perc'] = campaign_customer_features['campaign_total_saving'
                                                                    ] / campaign_customer_features['campaign_total_spend']
    campaign_customer_features['campaign_coupon_saving_frac'] = campaign_customer_features['campaign_coupon_saving'
                                                                    ] / campaign_customer_features['campaign_total_saving']
    campaign_customer_features['campaign_other_saving_frac'] = campaign_customer_features['campaign_other_saving'
                                                                    ] / campaign_customer_features['campaign_total_saving']
    campaign_customer_features['campaign_unique_item_purchased'] = df[['customer_id', 'item_id']].drop_duplicates().groupby(
                                                                            'customer_id').count().reset_index()['item_id']
    campaign_customer_features['campaign_unique_coupon_redeemed'] = df[['customer_id', 'coupon_id']].drop_duplicates().groupby(
                                                                            'customer_id').count().reset_index()['coupon_id']
    campaign_customer_features['campaign_unique_brand_purchased'] = df[['customer_id', 'brand']].drop_duplicates().groupby(
                                                                            'customer_id').count().reset_index()['brand']
    campaign_customer_features['campaign_most_frequently_purchased_item'] = df[['customer_id', 'item_id', 'date']].groupby(
                                                            ['customer_id', 'item_id']).count().reset_index()[['customer_id', 
                                                            'date']].groupby('customer_id').max().reset_index()['date']
    campaign_customer_features['campaign_most_frequently_purchased_brand'] = df[['customer_id', 'brand', 'date']].groupby(
                                                            ['customer_id', 'brand']).count().reset_index()[['customer_id', 
                                                            'date']].groupby('customer_id').max().reset_index()['date']
    campaign_customer_features['campaign_most_frequently_purchased_coupon_id'] = df1[['customer_id', 'coupon_id', 'date']].groupby(
                                                            ['customer_id', 'coupon_id']).count().reset_index()[['customer_id', 
                                                            'date']].groupby('customer_id').max().reset_index()['date']
    campaign_customer_features['campaign_mean_discount_perc_availed'] = df[['customer_id','total_discount_perc']].groupby(
                                                            'customer_id').mean().reset_index()['total_discount_perc']
    campaign_customer_features['campaign_best_deal_availed_frac'] = df[['customer_id','item_curr_deal_vs_best_deal']].groupby(
                                                            'customer_id').mean().reset_index()['item_curr_deal_vs_best_deal']
    df = df.sort_values(by = ['customer_id', 'date'])
    df['prev_date'] = df.date.shift(1)
    df['prev_date'][df.customer_id != df.customer_id.shift(1)] = None
    df['days_from_last_transaction'] = (df['date'] - df.prev_date).dt.days
    campaign_customer_features['campaign_transaction_frequency'] = df[['customer_id','days_from_last_transaction']].groupby(
                                                            'customer_id').mean().reset_index()['days_from_last_transaction']
    customer_category_wise_spend = df[['customer_id', 'category', 'selling_price']].groupby([
                            'customer_id', 'category']).sum().reset_index().pivot(index = 'customer_id',
                                columns = 'category',  values = 'selling_price')
    customer_category_wise_spend.columns = ["perc_spend_" + c for c in customer_category_wise_spend.columns]
    campaign_customer_features = campaign_customer_features.merge(customer_category_wise_spend.reset_index(), 
                                        on = 'customer_id', how = 'left').fillna(0)
    
    for v in customer_category_wise_spend.columns:
        campaign_customer_features[v] = campaign_customer_features[v] / campaign_customer_features.campaign_total_spend

   
    customer_category_wise_saving = df[['customer_id', 'category', 'total_discount']].groupby([
                                'customer_id', 'category']).sum().reset_index().pivot(index = 'customer_id',
                                columns = 'category', values = 'total_discount')
    customer_category_wise_saving.columns = ["perc_saving_" + c for c in customer_category_wise_saving.columns]
    campaign_customer_features = campaign_customer_features.merge(customer_category_wise_saving.reset_index(),
                                    on = 'customer_id', how = 'left').fillna(0)
    for v in customer_category_wise_saving.columns:
        campaign_customer_features[v] = campaign_customer_features[v] / campaign_customer_features.campaign_total_saving
    
    
    customer_category_wise_coupon_saving = df[['customer_id', 'category', 'coupon_discount']].groupby([
                                'customer_id', 'category']).sum().reset_index().pivot(index = 'customer_id', 
                                columns = 'category', values = 'coupon_discount')
    customer_category_wise_coupon_saving.columns = ["perc_coupon_saving_" + c for c in 
                                                    customer_category_wise_coupon_saving.columns]
    campaign_customer_features = campaign_customer_features.merge(customer_category_wise_coupon_saving.reset_index(), 
                                                                  on = 'customer_id', how = 'left').fillna(0)
    
    customer_brand_type_wise_coupon_saving = df[['customer_id', 'brand_type', 'coupon_discount']].groupby([
                        'customer_id', 'brand_type']).sum().reset_index().pivot(index = 'customer_id', columns = 'brand_type', 
                                    values = 'coupon_discount')
    customer_brand_type_wise_coupon_saving.columns = ["perc_coupon_saving_" + c for c in 
                                                      customer_brand_type_wise_coupon_saving.columns]
    campaign_customer_features = campaign_customer_features.merge(customer_brand_type_wise_coupon_saving.reset_index(), 
                                                                  on = 'customer_id', how = 'left').fillna(0)
    for v in customer_brand_type_wise_coupon_saving.columns:
        campaign_customer_features[v] = campaign_customer_features[v] / campaign_customer_features.campaign_coupon_saving
    
    campaign_transaction_data.append(campaign_customer_features)
    
      
    campaign_cust_coupon_features = df1[['customer_id', 'coupon_id', 'date']].groupby(['customer_id', 'coupon_id']
                                ).count().reset_index().rename(columns = {'date' : 'campaign_cust_coupon_transaction_count'})
    campaign_cust_coupon_features['campaign_id'] = row.campaign_id
    campaign_cust_coupon_features['cust_coupon_id_coupon_saving'] = df1[['customer_id', 'coupon_id', 'coupon_discount']].groupby(
                                                    ['customer_id', 'coupon_id']).sum().reset_index()['coupon_discount']
    campaign_cust_coupon_features['cust_coupon_item_curr_deal_vs_best_deal'] = df1[['customer_id', 'coupon_id', 'item_curr_deal_vs_best_deal']].groupby(
                                                    ['customer_id', 'coupon_id']).sum().reset_index()['item_curr_deal_vs_best_deal']
    
    customer_category_wise_coupon_saving = df1[['customer_id', 'coupon_id', 'category', 'coupon_discount']].groupby([
                                'customer_id', 'coupon_id', 'category']).sum().reset_index().pivot_table(index = ['customer_id',
                                'coupon_id'], columns = 'category', values = 'coupon_discount')
    customer_category_wise_coupon_saving.columns = ["perc_coupon_id_saving_" + c for c in 
                                                    customer_category_wise_coupon_saving.columns]
    campaign_cust_coupon_features = campaign_cust_coupon_features.merge(customer_category_wise_coupon_saving.reset_index(), 
                                                                  on = ['customer_id', 'coupon_id'], how = 'left').fillna(0)
    for v in customer_category_wise_coupon_saving.columns:
        campaign_cust_coupon_features[v] = campaign_cust_coupon_features[v] / campaign_cust_coupon_features.cust_coupon_id_coupon_saving
    
    customer_brand_type_wise_coupon_saving = df1[['customer_id','coupon_id', 'brand_type', 'coupon_discount']].groupby([
                        'customer_id','coupon_id', 'brand_type']).sum().reset_index().pivot_table(index = ['customer_id', 'coupon_id'], 
                                columns = 'brand_type', values = 'coupon_discount')
    customer_brand_type_wise_coupon_saving.columns = ["perc_coupon_id_saving_" + c for c in 
                                                      customer_brand_type_wise_coupon_saving.columns]
     
    campaign_cust_coupon_features = campaign_cust_coupon_features.merge(customer_brand_type_wise_coupon_saving.reset_index(), 
                                                                  on = ['customer_id', 'coupon_id'], how = 'left').fillna(0)
    for v in customer_brand_type_wise_coupon_saving.columns:
        campaign_cust_coupon_features[v] = campaign_cust_coupon_features[v] / campaign_cust_coupon_features.cust_coupon_id_coupon_saving
    
    days_since_last_coupon_redemption = df1[df1.coupon_discount < 0][['customer_id', 'coupon_id', 'date']].groupby(['customer_id',
                                                                                    'coupon_id']).max().date.to_dict()
    def get_days_since_last_coupon_redemption(cust, coupon):
        try:
            return (row.start_date - days_since_last_coupon_redemption[(cust, coupon)]).days
        except:
            return None
    campaign_cust_coupon_features['campaign_days_since_last_coupon_redemption_curr'] = [get_days_since_last_coupon_redemption(
                        row1.customer_id, row1.coupon_id) for row1 in campaign_cust_coupon_features.itertuples()]
    coupon_id_unique_cust_count = df1[['customer_id', 'coupon_id']].drop_duplicates().coupon_id.value_counts().to_dict()
    campaign_cust_coupon_features['coupon_id_unique_cust_count'] = campaign_cust_coupon_features.coupon_id.map(coupon_id_unique_cust_count)
    coupon_id_total_discount = df[['coupon_id', 'coupon_discount']].groupby('coupon_id').sum().coupon_discount.to_dict()
    campaign_cust_coupon_features['coupon_id_total_discount'] = campaign_cust_coupon_features.coupon_id.map(coupon_id_total_discount)
    coupon_mean_item_curr_deal_vs_best_deal = df1[['coupon_id', 'item_curr_deal_vs_best_deal']].groupby('coupon_id').mean().item_curr_deal_vs_best_deal.to_dict()
    campaign_cust_coupon_features['coupon_mean_item_curr_deal_vs_best_deal'] = campaign_cust_coupon_features.coupon_id.map(coupon_mean_item_curr_deal_vs_best_deal)
    
    campaign_cust_coupon_data.append(campaign_cust_coupon_features)
campaign_transaction_data = pd.concat(campaign_transaction_data)
campaign_cust_coupon_data = pd.concat(campaign_cust_coupon_data)

C:\Users\apurv\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\apurv\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\apurv\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

C:\Users\apurv\Anaconda3\lib\site-packages\ipykernel_launcher.py:165: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [14]:
coupon_categories = coupon_item_mapping.merge(item_data, on = 'item_id', how = 'left').sort_values(by = 'coupon_id')[[
                            'coupon_id', 'item_id', 'category']].groupby(['coupon_id', 'category']).count().reset_index().pivot(
                            index = 'coupon_id', columns = 'category', values = 'item_id').fillna(0)
coupon_categories.columns = ["coupon_num_items_in_" + c for c in coupon_categories.columns]

In [15]:
coupon_brand_type = coupon_item_mapping.merge(item_data, on = 'item_id', how = 'left').sort_values(by = 'coupon_id')[[
                            'coupon_id', 'item_id', 'brand_type']].groupby(['coupon_id', 'brand_type']).count().reset_index().pivot(
                            index = 'coupon_id', columns = 'brand_type', values = 'item_id').fillna(0)
coupon_brand_type.columns = ["coupon_num_items_in_" + c for c in coupon_brand_type.columns]
coupon_categories = coupon_categories.join(coupon_brand_type)

In [16]:
train1 = train.merge(campaign_transaction_data, on = ['customer_id', 'campaign_id'],
                     how = 'left').merge(campaign_data, on = 'campaign_id', 
            how = 'left').merge(customer_demographics, on = 'customer_id',
                                how = 'left').merge(coupon_categories.reset_index()
                                                    , on = 'coupon_id', how = 'left').merge(
            campaign_cust_coupon_data, on = ['customer_id', 'coupon_id', 'campaign_id']
    , how = 'left')

In [21]:
train1

,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_unique_coupon,campaign_unique_customer,campaign_customer_transaction_count,campaign_total_spend,campaign_total_saving,...,perc_coupon_id_saving_Natural Products,perc_coupon_id_saving_Packaged Meat,perc_coupon_id_saving_Pharmaceutical,perc_coupon_id_saving_Prepared Food,perc_coupon_id_saving_Restauarant,perc_coupon_id_saving_Salads,perc_coupon_id_saving_Seafood,perc_coupon_id_saving_Skin & Hair Care,perc_coupon_id_saving_Travel,perc_coupon_id_saving_Vegetables (cut)
0,1,13,27,1053,0,207,1077,270,45754.09,7971.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,13,116,48,0,207,1077,353,85211.51,10714.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6,9,635,205,0,18,176,769,91806.58,15702.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7,13,644,1050,0,207,1077,234,23028.54,4125.47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,8,1017,1489,0,208,1076,411,52827.50,9015.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78364,128587,8,71,1523,0,208,1076,361,41683.84,2826.34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78365,128589,30,547,937,0,178,361,141,11737.06,2044.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78366,128590,8,754,1004,0,208,1076,811,96378.09,11654.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78367,128592,13,134,71,0,207,1077,850,73150.69,20107.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
test1 = test.merge(campaign_transaction_data, on = ['customer_id', 'campaign_id'], how = 'left').merge(campaign_data, on = 'campaign_id', 
            how = 'left').merge(customer_demographics, on = 'customer_id', 
            how = 'left').merge(coupon_categories.reset_index(), on = 'coupon_id', how = 'left').merge(
            campaign_cust_coupon_data, on = ['customer_id', 'coupon_id', 'campaign_id'], how = 'left')